In [ ]:
#jiazai mokuai

#import os
#from google.colab import drive

#drive.mount('/content/drive') #挂载
#os.chdir("/content/drive/MyDrive/") # 更改运行目录

import gc
import pandas as pd
import numpy as np
!pip install scikit-learn
#导入分析库
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve, auc

!pip install 'numpy<1.24'
!pip uninstall -y protobuf # yes
!pip install open-clip-torch==2.20.0
!pip install protobuf==3.20.3
!pip install tensorflow==2.16.1 # for gpu

!pip install --upgrade pip
!pip install cmake
#import subprocess
#!pip install lightgbm --install-option=--use-wheel  #cpu
#!pip install lightgbm
!pip install lightgbm==3.3.2 #gpu
!pip install xgboost
!pip install catboost
import lightgbm as lgb
import xgboost as xgb
import catboost as cat

In [ ]:
# 读取数据
import pandas as pd
import zipfile
import os
os.chdir('./downloads/176950/')

with zipfile.ZipFile('user_log_format1.zip', 'r') as zip_ref:
    zip_ref.extractall()
data = pd.read_csv('user_log_format1.csv',nrows=1000, dtype={'time_stamp':'str'}) # 54925330 7
user_info = pd.read_csv('user_info_format1.csv',nrows=1000) # 424170 3
train_format1 = pd.read_csv('train_format1.csv',nrows=1000) # 260864 3
submission = pd.read_csv('test_format1.csv',nrows=1000) # 261477 3
data_train = pd.read_csv('train_format2.csv',nrows=1000) # 7030723 6

In [ ]:
data.rename(columns={'seller_id':'merchant_id'}, inplace=True)
train_format1['origin'] = 'train' # 添加新列'origin'，设置该列值全为'train'
submission['origin'] = 'test' # 添加新列'origin'，设置该列值全为'test'
matrix = pd.concat([train_format1, submission], ignore_index=True, sort=False) # 垂直合并
matrix.drop(['prob'], axis=1, inplace=True)
matrix = matrix.merge(user_info, on='user_id', how='left') # 水平合并
#print(matrix)
del user_info, train_format1
gc.collect()

In [ ]:
# 格式化
data['user_id'] = data['user_id'].astype('int32')
data['merchant_id'] = data['merchant_id'].astype('int32')
data['item_id'] = data['item_id'].astype('int32')
data['cat_id'] = data['cat_id'].astype('int32')
data['brand_id'].fillna(0, inplace=True)
data['brand_id'] = data['brand_id'].astype('int32')
current_year = 2022
# 新的日期字符串列，包含年份信息
data['time_stamp'] = data['time_stamp'].apply(lambda x: f"{current_year}{x}")
data['time_stamp'] = pd.to_datetime(data['time_stamp'], format='%Y%m%d', errors='coerce')

# 缺失值填充
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'].fillna(matrix['age_range'].median(),inplace=True) # 中位数填充

# 格式化
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')

In [ ]:
# 特征处理
# User特征处理
groups = data.groupby(['user_id'])
# 用户交互行为数量
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
# 统计唯一值个数
# item_id
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left') # 合并
# cat_id
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left') # 合并
#merchant_id
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left') # 合并
# brand_id
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left') # 合并
# 时间间隔天数
temp = data.groupby('user_id')['time_stamp'].agg(['min', 'max']).reset_index()
temp['u6'] = (temp['max'] - temp['min']).dt.days
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')


# 统计操作类型为0，1，2，3的个数
temp = groups['action_type'].value_counts().unstack().reset_index().rename(
    columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')
#print(matrix)
del temp # 释放临时变量
gc.collect()

In [ ]:
# 商家特征处理
groups = data.groupby(['merchant_id'])
# 商家被交互行为数量
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = groups.agg({'user_id': 'nunique','item_id': 'nunique',
                   'cat_id': 'nunique','brand_id': 'nunique'
}).reset_index().rename(columns={
    'user_id':'m2','item_id':'m3',
    'cat_id':'m4','brand_id':'m5'
})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的action_type 唯一值
temp = groups['action_type'].value_counts().unstack().reset_index().rename(
    columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

del temp
gc.collect()

In [ ]:
# 按照merchant_id 统计随机负采样的个数
# 从名为data_train的数据集中筛选出所有标签为-1的行。在许多二元分类问题中，-1通常表示负类（例如，用户未购买或未点击
# 这段代码的目的是为每个商家在matrix数据框中增加一个特征，表示该商家的负样本数量，这对于评估商家的整体表现或进行特征工程可能很有用。
temp = data_train[data_train['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

In [ ]:
# 用户+商户特征
# 按照用户ID和商户ID对数据进行分组
groups = data.groupby(['user_id', 'merchant_id'])
# 每个用户与商户组合的交互次数
temp = groups.size().reset_index().rename(columns={0:'um1'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
# 统计用户+商户交互的item_id, cat_id, brand_id 唯一值 # 或称独特值
temp = groups[['item_id', 'cat_id', 'brand_id']].nunique().reset_index().rename(columns={
    'item_id':'um2','cat_id':'um3','brand_id':'um4'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
# 每个用户-商户组合中不同action_type（可能代表用户行为类型）的频率
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={
    0:'um5',1:'um6',2:'um7',3:'um8'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
# 用户+商户交互时间范围（最早和最晚）以及时间差
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'].dt.floor('d') - temp['first'].dt.floor('d')).dt.days  # 计算时间差（天）
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')

del temp
gc.collect()

In [ ]:
matrix['r1'] = matrix['u9']/matrix['u7'] # 用户购买点击比
matrix['r2'] = matrix['m8']/matrix['m6'] # 商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5'] #不同用户不同商家购买点击比


matrix.fillna(0, inplace=True)

In [ ]:
# 行独热编码
# age_range和gender列转换为独热编码的形式，新生成列添加回原有的DataFrame中。这有助于后续的机器学习模型训练和预测
# 修改age_range字段名称为 age_0, age_1, age_2... age_8
#print(matrix)
temp = pd.get_dummies(matrix['age_range'], prefix='age') # 前缀
matrix = pd.concat([matrix, temp], axis=1) # axis=0 垂直合并 axis=1 水平合并
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)
#print(matrix)
del temp
gc.collect()

In [ ]:
# 原始数据集 matrix 中标记为 'train' 的部分
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
# 原始数据集 matrix 中标记为 'test' 的部分
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
# 写入csv文件
train_data.to_csv('../train_data.csv',index=False)
test_data.to_csv('../test_data.csv',index=False)

del matrix
gc.collect()

In [ ]:
# 训练数据部分



train_data = pd.read_csv('../train_data.csv')
test_data = pd.read_csv('../test_data.csv')
# train_data = train_data[train_col] # train_col是一个列表，包含了你想要从train_data中选择的列名
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

del train_data

X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.2, random_state=42) # test_size=.3

In [ ]:
#XGB Model

train_data = pd.read_csv('../train_data.csv')

train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']
del train_data
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.2, random_state=42) # test_size=.3


In [ ]:
def xgb_train(X_train, y_train, X_valid, y_valid, verbose=True):
    model_xgb = xgb.XGBClassifier(
        max_depth=10, # raw8
        n_estimators=1000,
        min_child_weight=300,
        colsample_bytree=0.8,
        subsample=0.8,
        eta=0.3,
        seed=42
    )

    model_xgb.fit(
        X_train,
        y_train,
        eval_metric='auc',
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        verbose=verbose,
        early_stopping_rounds=10 # 早停法，如果auc在10epoch没有进步就stop
    )
    print(model_xgb.best_score)
    return model_xgb

In [ ]:
from xgboost import XGBClassifier

# 参数字典
params = {'eval_metric': 'auc', 'early_stopping_rounds': 10, }

# 创建模型
model_xgb = XGBClassifier(**params)

# 训练模型
model_xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)

In [ ]:
prob = model_xgb.predict_proba(test_data)

submission['prob'] = pd.Series(prob[:,1])
#submission.drop(['origin'], axis=1, inplace=True) # test
submission.to_csv('../submission_xgb.csv', index=False)

In [ ]:
#LGB Model

############DEF:lgb_train################
import lightgbm
from lightgbm import log_evaluation, early_stopping
callbacks = [log_evaluation(period=100), early_stopping(stopping_rounds=30)]

def lgb_train(X_train, y_train, X_valid, y_valid, verbose=True):
    model_lgb = lgb.LGBMClassifier(
        max_depth=10, # 8
        n_estimators=1000,
        min_child_weight=200,
        colsample_bytree=0.8,
        subsample=0.8,
        seed=42,
    )
    model_lgb.fit(
        X_train,
        y_train,
        eval_metric='auc',
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        callbacks=callbacks
    )

    print(model_lgb.best_score_['valid_1']['auc'])
    return model_lgb

In [ ]:
model_lgb = lgb_train(X_train, y_train, X_valid, y_valid, verbose=False)

In [ ]:
prob = model_lgb.predict_proba(test_data)
submission['prob'] = pd.Series(prob[:,1])
submission.to_csv('../submission_lgb.csv', index=False)

In [ ]:
#Cat Model

def cat_train(X_train, y_train, X_valid, y_valid, verbose=True):
    model_cat = cat.CatBoostClassifier(learning_rate=0.02, iterations=5000, eval_metric='AUC', od_wait=50,
                                od_type='Iter', random_state=10, thread_count=8, l2_leaf_reg=1, verbose=verbose)
    model_cat.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=50,
            use_best_model=True)

    print(model_cat.best_score_['validation']['AUC'])
    return model_cat

In [ ]:
model_cat = cat_train(X_train, y_train, X_valid, y_valid, verbose=False)

In [ ]:
prob = model_cat.predict_proba(test_data)
submission['prob'] = pd.Series(prob[:,1])
submission.to_csv('../submission_cat.csv', index=False)

In [ ]:
#StratifiedKFold

# 构造训练集和测试集
def get_train_testDF(train_df,label_df):
    skv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    trainX = []
    trainY = []
    testX = []
    testY = []
    for train_index, test_index in skv.split(X=train_df, y=label_df):
        train_x, train_y, test_x, test_y = train_df.iloc[train_index, :], label_df.iloc[train_index], \
                                            train_df.iloc[test_index, :], label_df.iloc[test_index]

        trainX.append(train_x)
        trainY.append(train_y)
        testX.append(test_x)
        testY.append(test_y)
    return trainX, testX, trainY, testY

In [ ]:
#lightgbm

train_data = pd.read_csv('../train_data.csv')
test_data = pd.read_csv('../test_data.csv')

train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

del train_data

# Split Train&Valid Data
X_train, X_valid, y_train, y_valid = get_train_testDF(train_X, train_y)

In [ ]:
# 将训练数据集划分分别训练5个lgbm,xgboost和catboost 模型
# lightgbm模型
from lightgbm import log_evaluation, early_stopping
callbacks = [log_evaluation(period=100), early_stopping(stopping_rounds=30)]
pred_lgbms = []
for i in range(5):
    print('\n============================LGB training use Data {}/5============================\n'.format(i+1))
    model_lgb = lgb.LGBMClassifier(
        max_depth=10, # 8
        n_estimators=1000,
        min_child_weight=200,
        colsample_bytree=0.8,
        subsample=0.8,
        eta=0.3,
        seed=42
    )

    model_lgb.fit(
        X_train[i],
        y_train[i],
        eval_metric='auc',
        eval_set=[(X_train[i], y_train[i]), (X_valid[i], y_valid[i])],
        callbacks=callbacks
        )

    print(model_lgb.best_score_['valid_1']['auc'])

    pred = model_lgb.predict_proba(test_data)
    pred = pd.DataFrame(pred[:,1])
    pred_lgbms.append(pred)
pred_lgbms = pd.concat(pred_lgbms, axis=1)
print(pred_lgbms)

submission['prob'] = pred_lgbms.mean(axis=1)
# submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('../submission_KFold_lgb.csv', index=False)

####0.6784

In [ ]:
#catgbm

train_data = pd.read_csv('../train_data.csv')
test_data = pd.read_csv('../test_data.csv')

train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

del train_data

# Split Train&Valid Data
X_train, X_valid, y_train, y_valid = get_train_testDF(train_X, train_y)

In [ ]:
# 将训练数据集划分分别训练5个lgbm,xgboost和catboost 模型
# catgbm模型

pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.02, iterations=5000, eval_metric='AUC', od_wait=50,
                                od_type='Iter', random_state=10, thread_count=8, l2_leaf_reg=1, verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50,
            use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])

    pred = model_cat.predict_proba(test_data)
    pred = pd.DataFrame(pred[:,1])
    pred_cats.append(pred)
pred_cats = pd.concat(pred_cats, axis=1)

submission['prob'] = pred_cats.mean(axis=1)
# submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('../submission_KFold_cat.csv', index=False)


#### 0.68001

In [ ]:
#xgboost

train_data = pd.read_csv('../train_data.csv')
test_data = pd.read_csv('../test_data.csv')

train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

del train_data

# Split Train&Valid Data
X_train, X_valid, y_train, y_valid = get_train_testDF(train_X, train_y)

In [ ]:
# 将训练数据集划分分别训练5个lgbm,xgboost和catboost 模型
# xgboost模型
# 从较小的早停轮数开始，比如10、20或50轮，这取决于你的数据集大小和复杂度。这些初始值可以作为起点，帮助你了解模型在何时开始过拟合
pred_xgbs = []
for i in range(5):
    print('\n============================XGB training use Data {}/5============================\n'.format(i+1))
    model_xgb = xgb.XGBClassifier(
        max_depth=10, # raw8
        n_estimators=1000,
        min_child_weight=300,
        colsample_bytree=0.8,
        subsample=0.8,
        eta=0.3,
        seed=42,
        eval_metric='auc',
        verbose=False,
        early_stopping_rounds=10 # 早停法，如果auc在10epoch没有进步就stop
    )

    model_xgb.fit(
        X_train[i],
        y_train[i],
        eval_set=[(X_train[i], y_train[i]), (X_valid[i], y_valid[i])],
    )

    print(model_xgb.best_score)

    pred = model_xgb.predict_proba(test_data)
    pred = pd.DataFrame(pred[:,1])
    pred_xgbs.append(pred)
pred_xgbs = pd.concat(pred_xgbs, axis=1)

# make submission
submission['prob'] = pred_xgbs.mean(axis=1)
# submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('../submission_KFold_xgb.csv', index=False)

#### 0.6803

In [ ]:
#xgb:0.689278, ##KFold## 0.6784
#lgb:0.688217, ##KFold## 0.6800
#cat:0.688843, ##KFold## 0.6803

In [ ]:
#blending


lgb6812 = pd.read_csv("../submission_KFold_lgb.csv")
xgb6787 = pd.read_csv("../submission_KFold_xgb.csv")
cat6777 = pd.read_csv("../submission_KFold_cat.csv")

In [ ]:
# 先构造一个矩阵
df = np.array([lgb6812.prob, xgb6787.prob, cat6777.prob])
# 计算协方差矩阵
np.corrcoef(df)

In [ ]:
sub = lgb6812.copy()

#sub.prob = 0.6*lgb6812.prob + 0.4*cat6777.prob # Online test score:0.6830807
#sub.to_csv('./sub_blended11.csv', index=False)
####################################0.6833209################################
#sub.prob = 0.5*lgb6812.prob + 0.3*cat6777.prob + 0.2*xgb6787.prob# Online test 0.6833209
#sub.to_csv('./sub_blended12.csv', index=False)

#sub.prob = 0.45*lgb6812.prob + 0.3*cat6777.prob + 0.25*xgb6787.prob# Online test 0.6832934
#sub.to_csv('./sub_blended13.csv', index=False)
####################################0.6833171################################
sub.prob = 0.45*lgb6812.prob + 0.35*cat6777.prob + 0.2*xgb6787.prob# Online test 0.6833171
sub.to_csv('../sub_blended14.csv', index=False)